# How to run:

*   **Libraries**, to import all the needed libraries (if Colab shows an error, restart the runtime).
*   If you would create the toy graph,
    * **Toy Graph (with two communities)**, to create the graph with NetworkX
*   Otherwise,
    * **Import Graph from dataset**, to import one of the PyKEEN public datasets (Nations, Kinships, UMLS, FB15k-237, WN18, ecc.)
    * **Create and visualize the KG**, to create the graph (from the chosen dataset) with NetworkX. You can decide to make it *undirected* or *directed*, commenting the proper line of code.
*   If you already have a trained model,
    * **Load model**, to load a pre-trained model on the chosen dataset.
*   Otherwise,
    *   **Divide into three groups**, to separate the triplets in *Positive*, *Negative* and *Different relation* sets. The last one gets all the triplets generated from two neighboring entities, differently from the Negative set, which contains all the triplets from two entities that have no linked relations between them. Run this task only if you want to get information about distribution of scores for each set.
    *   **Train the embeddings**, to train the embeddings on the training set retrieved from the chosen dataset.
*   **Extract the embeddings**, to achieve entity/relation embeddings using the newly trained model.
*   **Reliability**, to define the function used to compute the reliability score on a set of triplets in three different ways: *Normalized Score*, *Non Normalized Score*, *Sigmoid Score*.
*   **LP Classifier**, to define the function used to create a classifier with training/test sets composed by triplets in the selected subgraph (or also for the entire graph).
*   **Reliability/Accuracy of subgraph**, functions mentioned above are called, passing subgraphs generated by a random walk on the original graph. Make sure to use the correct version (*Undirected* or *Directed*), based on the choice made in the initial part of the process.
*   **Reliability/Accuracy of the graph**, in this way both functions will compute their values with the given information on the entire graph.
*   **Compute scores**, to compute scores of every single triple in the three generated sets.
*   **Plot scores**, to plot distributions of scores.

The last part (evaluation) regards the ranking, so it is possible to compute metrics on the information retrieved by the ranking of triplets between every couple of entities in the graph.

# Libraries

In [ ]:
# Install packages if they are not already found
! python -c "import pykeen" || pip install git+https://github.com/pykeen/pykeen.git
! python -c "import networkx" || pip install networkx

In [ ]:
from pkg_resources import require
require('pykeen')

# Import libraries
import os
from google.colab import drive

import matplotlib.pyplot as plt
import scipy.stats as st
import torch

import pykeen
from pykeen.pipeline import pipeline
from pykeen.datasets import get_dataset
from pykeen.triples import TriplesFactory
from pykeen.models.predict import *

import networkx as nx
import pandas as pd
import csv

import numpy as np
import seaborn as sns

import math
import random
import itertools
import statistics
import time

# Access data from Google Drive
drive.mount('/content/drive', force_remount='True')
drive_path = '/content/drive/My Drive/KG_Reliability_Models/'

# Change the model
my_model = 'TransE'

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

Mounted at /content/drive


In [ ]:
pykeen.env()

# Toy Graph (with two communities)

In [ ]:
dataset = "ToyGraph"

nodes = list(range(101))

relations = ['a', 'b']

a_edges = list()
b_edges = list()

a = nx.Graph()

a.add_nodes_from(nodes)

for n1 in range(0, len(nodes)):
    for n2 in range(n1 + 1, len(nodes)):
        a.add_edge(n1, n2, label = 'a')
        if n1 == 0:
            a_edges.append((str(n1), 'a', 'a-' + str(n2)))
        else:
            a_edges.append(('a-' + str(n1), 'a', 'a-' + str(n2)))

b = nx.Graph()

b.add_nodes_from(nodes)

for n1 in range(0, len(nodes)):
    for n2 in range(n1 + 1, len(nodes)):
        b.add_edge(n1, n2, label = 'b')
        if n1 == 0:
            b_edges.append((str(n1), 'b', 'b-' + str(n2)))
        else:
            b_edges.append(('b-' + str(n1), 'b', 'b-' + str(n2)))

a = nx.relabel_nodes(a, { n: str(n) if n==0 else 'a-' + str(n) for n in nodes })
b = nx.relabel_nodes(b, { n: str(n) if n==0 else 'b-' + str(n) for n in nodes })

G = nx.compose(a, b)

c_edges = a_edges + b_edges

print(len(a_edges))

tf = TriplesFactory.from_labeled_triples(np.array(c_edges))

print(len(tf.triples))

train_triples, test_triples = tf.split()

nx.draw_networkx(G, with_labels=True, node_size=500)
plt.show()

# Import Graph from dataset

In [ ]:
# Get all the triples of the KG

dataset = "WN18" # Kinships, UMLS, FB15k-237, WN18

tf_train = get_dataset(dataset=dataset).training.triples.tolist()
tf_test  = get_dataset(dataset=dataset).testing.triples.tolist()
tf_valid  = get_dataset(dataset=dataset).validation.triples.tolist()

tf_all = tf_train + tf_test + tf_valid

relations = []

for t in tf_all:
    if t[1] not in relations: # To limit relations number to 5, add " and len(relations) < 5"
        relations.append(t[1])

print("Relations: ", relations)

tf = TriplesFactory.from_labeled_triples(np.array(tf_all))

print("Total triples: ", len(tf.triples))

# Split the input KG into Train and LP+
#train, LP_pos = tf.split([.5, .5])

# Split the KG into Train and Test (70/30) for the training phase
train_triples, test_triples = tf.split()

# Create and visualize the KG

In [ ]:
ds = tf.triples

print(type(ds))

df = pd.DataFrame(ds, columns = ['subject', 'predicate', 'object'])

# Create an undirected graph class that can store multiedges
#G = nx.MultiGraph()

# Create a directed graph class that can store multiedges
G = nx.MultiDiGraph()

for t in df.values:
    G.add_edge(t[0], t[2], label = t[1])

entities = list(G.nodes())

# Info and plot KG
'''
print(nx.info(G))
print(type(G))
print(G.nodes)
print('Total triples in the KG:', ds.shape)
print([val for (node, val) in G.degree()])

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()
'''

# Load model

In [ ]:
model = torch.load(drive_path + my_model + '/' + dataset + '/trained_model.pkl') # Insert the name of a trained model

print(model)

# Divide into three groups
###### (If you want to get information about distributions)



In [ ]:
# To obtain the running time of this task
start_time = time.time()

def nodes_connected(u, v):
    return u in G.neighbors(v)

entities  = list(G.nodes())
#relations = ['a', 'b'] # Only for the toy graph, comment if you use a dataset

for t in train_triples.triples:
    if t[0] not in entities:
        entities.append(t[0])

# Existing triples in the KG
t = tf.triples.tolist()

# Positive triples with fixed relations
pos_triples = t

# Only the relation is changed in an existing triple between two nodes
diff_rel_triples = []

# Non existing triples in the KG (no triples between two nodes)
neg_triples = []

# Scores of all triples
score_pos       = []
score_diff_rel  = []
score_neg       = []

c = 0

for x in range(0, len(entities)):
    for y in range(x + 1, len(entities)):
        # List to store a relation if exists in positive examples
        curr_rel        = []
        # All combinations between a selected couple of entities
        e_combinations  = np.column_stack([
                [entities[x]] * len(relations),
                relations,
                [entities[y]] * len(relations)]).tolist()

        #print(e_combinations)
        #for r in relations:
            #curr_list = [entities[x], r, entities[y]]
            #if nodes_connected(curr_list[0], curr_list[2]) and curr_list not in pos_triples:
            #    curr_rel.append(curr_list)
            #e_combinations.append(curr_list)

        for i in range(0, len(e_combinations)):
            if e_combinations[i] not in pos_triples:
                if not nodes_connected(e_combinations[i][0], e_combinations[i][2]) and not nodes_connected(e_combinations[i][2], e_combinations[i][0]):
                    neg_triples.append(e_combinations[i])
                else:
                    diff_rel_triples.append(e_combinations[i])
                neg_triples.append(e_combinations[i])
                
            
            c += 1
            #print(c, len(pos_triples), len(neg_triples))
            #if len(neg_triples) == 10000:
            #    break

        '''
        for i in range(0, len(e_combinations)):
            if not nodes_connected(e_combinations[i][0], e_combinations[i][2]):
                neg_triples.append(e_combinations[i])
            elif e_combinations[i] not in pos_triples and [e_combinations[i][2], e_combinations[i][1], e_combinations[i][0]] not in pos_triples:
                diff_rel_triples.append(e_combinations[i])
            else:
                pos_triples.append(e_combinations[i])

            c += 1
        '''

#print(len(pos_triples))    
#print(len(neg_triples[:len(pos_triples)]))        
                    
all_triples = pos_triples + diff_rel_triples + neg_triples

tf_all_triples = TriplesFactory.from_labeled_triples(np.array(all_triples))

print("Positive: " + str(len(pos_triples)) + 
      ", Different relation: " + str(len(diff_rel_triples)) + 
      ", Negative: " +  str(len(neg_triples)) + "\n")

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
## GENERATE NEGATIVE SAMPLES (it is not necessary anymore) ##

'''

import numpy as np

entities  = []
relations = []

for t in train_triples.triples:
    if t[0] not in entities:
        entities.append(t[0])
    if t[1] not in relations:
        relations.append(t[1])

print(entities, relations)

all_triples     = []
#all_triples_rev = []

for r in range(len(relations)):
    for e in range(len(entities)):
        for x in range(e+1, len(entities)):
            all_triples.append([entities[e], relations[r], entities[x]])
        for y in range(len(entities)):
            all_triples_rev.append([entities[e], relations[r], entities[y]])

all_triples = list(itertools.combinations(itertools.chain(entities, relations, entities), 3))

print(len(all_triples))

positive_samples = (LP_pos.triples).tolist()
negative_samples = []

for t in all_triples:
    if t not in positive_samples:
        negative_samples.append(t)

LP_all = np.array(all_triples)

print(len(LP_all), len(LP_pos.triples), len(LP_neg))


#LP_all_rev  = np.array(all_triples_rev)

LP_neg = np.array(negative_samples)

'''

In [ ]:
## SPLIT AND MERGE TRAINING/TEST SETS FOR LP TASK (it is not necessary anymore) ##

'''

LP_neg = np.array(diff_rel_triples + neg_triples)

trainLP_pos = tf.triples[:int(len(tf.triples)*0.7)]
testLP_pos = tf.triples[int(len(tf.triples)*0.7):]

trainLP_neg = LP_neg[:int(len(LP_neg)*0.7)]
testLP_neg  = LP_neg[int(len(LP_neg)*0.7):]

trainfLP_pos = TriplesFactory.from_labeled_triples(np.array(trainLP_pos)) 
trainfLP_neg = TriplesFactory.from_labeled_triples(np.array(trainLP_neg)) 

train_LP = trainLP_pos.tolist()
for l in trainLP_neg.tolist():
    train_LP.append(l)

trainf_LP = TriplesFactory.from_labeled_triples(np.array(train_LP)) 

test_LP = testLP_pos.tolist()

for l in testLP_neg.tolist():
    test_LP.append(l)

testf_LP = TriplesFactory.from_labeled_triples(np.array(test_LP))

print(len(train_LP), len(test_LP))

'''

# Train the embeddings

In [ ]:
pipeline_result = pipeline(
    training=train_triples,
    testing=test_triples,
    model=my_model,
    training_loop='sLCWA',
    negative_sampler='basic',
    negative_sampler_kwargs=dict(
        filtered=True,    
    ),
    # Training configuration
    training_kwargs=dict(
        num_epochs=200,
        use_tqdm_batch=False,
    ),  
    # Runtime configuration
    random_seed=1235,
    device='cpu',
)

pipeline_result.save_to_directory(drive_path + my_model + '/' + dataset)

model = pipeline_result.model

# Extract embeddings

In [ ]:
# Extract entity embeddings
entity_ids = torch.LongTensor(range(tf.num_entities))
e_emb = model.entity_embeddings
e_emb.cpu()
e_emb_numpy = e_emb(entity_ids).detach().numpy()
entity2embedding = {}
for eid in range(tf.num_entities):
  e = tf.entity_id_to_label[eid]
  entity2embedding[e] = list(e_emb_numpy[eid])

# Extract relation embeddings
relation_ids = torch.LongTensor(range(tf.num_relations))
r_emb = model.relation_embeddings
r_emb.cpu()
r_emb_numpy = r_emb(relation_ids).detach().numpy()
relation2embedding = {}
for rid in range(tf.num_relations):
  r = tf.relation_id_to_label[rid]
  relation2embedding[r] = list(r_emb_numpy[rid])

# Reliability

In [ ]:
# Function to reject outliers
def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

# Compute the sigmoid on top the scores
def sigmoid(x, max_score, min_score):

    # Normalize x between -6 and 6
    # x' = (b - a) * (x – min(x)) / (max(x) – min(x)) + a
    x_norm = 12 * (x - min_score) / (max_score - min_score) - 6

    # Apply the sigmoid
    return 1 / (1 + math.exp(-x_norm))

# Check if two nodes are connected
def nodes_connected(u, v):
    return u in G.neighbors(v)

# Compute the reliability measure    
def reliability(G, pos_triples, diff_rel_triples, neg_triples):

    # To obtain the running time of this task
    start_time = time.time()

    print("Positive: ", len(pos_triples))
    print("Negative: ", len(diff_rel_triples))
    print("Different relation: " + str(len(neg_triples)) + "\n")

    tot_triples = pos_triples + diff_rel_triples + neg_triples

    sum_tot      = 0 # Sum of all scores
    sum_tot_norm = 0 # Sum of all normalized scores
    sum_tot_sigm = 0 # Sum of all scores applying a sigmoid on top

    sum_tot_norm_nodiff = 0 # Sum of all normalized scores (without diff_rel)

    sum_tot_norm_2 = 0 # Normalized scores added up at the end
    sum_tot_sigm_2 = 0 # Sigmoid scores added up at the end

    sum_tot_norm_3 = 0 # Normalized scores with mean and standard deviation

    # Normalized Reliability
    sum_norm_pos      = 0
    sum_norm_diff_rel = 0
    sum_norm_neg      = 0

    # Sigmoid Reliability
    sum_sigm_pos      = 0
    sum_sigm_diff_rel = 0
    sum_sigm_neg      = 0

    # Single set score
    score_pos       = []
    score_diff_rel  = []
    score_neg       = []

    new_scores = []

    all_scores = model.score_hrt(TriplesFactory.from_labeled_triples(np.array(tot_triples)).mapped_triples)

    max_score = max(all_scores.detach().numpy())[0]
    min_score = min(all_scores.detach().numpy())[0]

    print("Mean and std before: " + str(np.mean(all_scores.tolist())) + ", " + str(np.std(all_scores.tolist())))
    print(max_score, min_score)

    new_scores = reject_outliers(all_scores.detach().numpy())

    max_score = max(new_scores)
    min_score = min(new_scores)
    
    print("Mean and std after: " + str(np.mean(new_scores.tolist())) + ", " + str(np.std(new_scores.tolist())))
    print(max_score, min_score)

    # Normalize vectors
    # zi = (xi – min(x)) / (max(x) – min(x)), where 
    # xi is the current value
    # x is the array of scores

    if len(pos_triples) > 0:
        for i in range(0, len(pos_triples)):   
            score_triple = model.score_hrt(torch.LongTensor([[tf.entity_to_id[pos_triples[i][0]], 
                                                                              tf.relation_to_id[pos_triples[i][1]], 
                                                                              tf.entity_to_id[pos_triples[i][2]]]]))
            score = score_triple.detach().numpy()
            if score > max_score:
                score_norm = 1
            elif score < min_score:
                score_norm = 0
            else:
                score_norm = (score - min_score) / (max_score - min_score)

            score_sigm   = sigmoid(score, max_score, min_score)
            score_pos.append(score)
            sum_tot      += float(score)
            sum_tot_norm += float(score_norm)
            sum_tot_sigm += float(score_sigm)
            sum_tot_norm_nodiff += float(score_norm)
            sum_norm_pos += float(score_norm)
            sum_sigm_pos += float(score_sigm)

    if len(diff_rel_triples) > 0:
        for i in range(0, len(diff_rel_triples)):
            score_triple = model.score_hrt(torch.LongTensor([[tf.entity_to_id[diff_rel_triples[i][0]], 
                                                                              tf.relation_to_id[diff_rel_triples[i][1]], 
                                                                              tf.entity_to_id[diff_rel_triples[i][2]]]]))
            score = score_triple.detach().numpy()
            if score > max_score:
                score_norm = 1
            elif score < min_score:
                score_norm = 0
            else:
                score_norm = (score - min_score) / (max_score - min_score)

            score_sigm        = sigmoid(score, max_score, min_score)
            score_diff_rel.append(score)
            sum_tot           += float(score)
            sum_tot_norm      += 1 - float(score_norm)
            sum_tot_sigm      += 1 - float(score_sigm)
            sum_norm_diff_rel += 1 - float(score_norm)
            sum_sigm_diff_rel += 1 - float(score_sigm)

    if len(neg_triples) > 0:
        for i in range(0, len(neg_triples)):
            score_triple = model.score_hrt(torch.LongTensor([[tf.entity_to_id[neg_triples[i][0]], 
                                                                              tf.relation_to_id[neg_triples[i][1]], 
                                                                              tf.entity_to_id[neg_triples[i][2]]]]))
            score = score_triple.detach().numpy()
            if score > max_score:
                score_norm = 1
            elif score < min_score:
                score_norm = 0
            else:
                score_norm = (score - min_score) / (max_score - min_score)

            score_sigm   = sigmoid(score, max_score, min_score)
            score_neg.append(score)
            sum_tot      += float(score)
            sum_tot_norm += 1 - float(score_norm)
            sum_tot_sigm += 1 - float(score_sigm)
            sum_tot_norm_nodiff += 1 - float(score_norm)
            sum_norm_neg += 1 - float(score_norm)
            sum_sigm_neg += 1 - float(score_sigm)


    # Mean and standard deviation of the subgraph
    subgraph_mean = np.mean(sum_tot)
    subgraph_std  = np.std(sum_tot)

    # Mean and standard deviation of the single sets
    # (Positive, Different relation, Negative)
    pos_mean = np.mean(score_pos)
    pos_std  = np.std(score_pos)

    diff_rel_mean = np.mean(score_diff_rel)
    diff_rel_std  = np.std(score_diff_rel)

    neg_mean = np.mean(score_neg)
    neg_std  = np.std(score_neg)

    score_pos = (score_pos - pos_mean) / pos_std

    sum_tot      = sum_tot / (len(pos_triples) + len(diff_rel_triples) + len(neg_triples))
    sum_tot_norm = sum_tot_norm / (len(pos_triples) + len(diff_rel_triples) + len(neg_triples))
    sum_tot_sigm = sum_tot_sigm / (len(pos_triples) + len(diff_rel_triples) + len(neg_triples))

    sum_tot_norm_nodiff = sum_tot_norm_nodiff / (len(pos_triples) + len(diff_rel_triples) + len(neg_triples))

    # Number of non-negative sets
    n = 0

    if(len(pos_triples)):
        sum_norm_pos      /= len(pos_triples)
        sum_sigm_pos      /= len(pos_triples)
        n += 1
    if(len(diff_rel_triples)):
        sum_norm_diff_rel /= len(diff_rel_triples)
        sum_sigm_diff_rel /= len(diff_rel_triples)
        n += 1
    if(len(neg_triples)):
        sum_norm_neg      /= len(neg_triples)
        sum_sigm_neg      /= len(neg_triples)
        n += 1

    sum_tot_norm_2 = (sum_norm_pos + sum_norm_diff_rel + sum_norm_neg) / n
    sum_tot_sigm_2 = (sum_sigm_pos + sum_sigm_diff_rel + sum_sigm_neg) / n

    rel_scores.append(float("{:.3f}".format(sum_tot)))
    rel_scores_norm.append(float("{:.3f}".format(sum_tot_norm)))
    rel_scores_sigm.append(float("{:.3f}".format(sum_tot_sigm)))
    rel_scores_sigm_end.append(float("{:.3f}".format(sum_tot_sigm_2)))

    print("Reliability of subgraph              -> %.2f" % sum_tot)
    print("Normalized Reliability of subgraph   -> %.2f" % sum_tot_norm)
    print("Sigmoid Reliability of subgraph      -> %.2f" % sum_tot_sigm + "\n")

    print("Normalized Reliability of subgraph (without diff_rel)   -> %.2f" % sum_tot_norm_nodiff)
    print("Normalized Reliability (end)         -> %.2f" % sum_tot_norm_2)
    print("Sigmoid Reliability (end)            -> %.2f" % sum_tot_sigm_2 + "\n")


    print("--- %s seconds RELIABILITY ---\n" % (time.time() - start_time))


# Total reliability of the KG (TODO)
'''
def total_reliability(G, pos_triples, diff_rel_triples, neg_triples):

    score_pos       = []
    score_diff_rel  = []
    score_neg       = []

    print("Positive: ", len(pos_triples))
    print("Negative: ", len(diff_rel_triples))
    print("Different relation: " + str(len(neg_triples)) + "\n")

    tot_triples = pos_triples + diff_rel_triples + neg_triples

    sum_tot      = 0 # Sum of all scores
    sum_tot_norm = 0 # Sum of all normalized scores

    all_scores = pipeline_result.model.score_hrt(TriplesFactory.from_labeled_triples(np.array(tot_triples)).mapped_triples)
    max_score = max(all_scores.detach().numpy())[0]
    min_score = min(all_scores.detach().numpy())[0]

    if len(pos_triples) > 0:
        for i in range(0, len(pos_triples)):   
            score_triple = pipeline_result.model.score_hrt(torch.LongTensor([[tf.entity_to_id[pos_triples[i][0]], 
                                                                              tf.relation_to_id[pos_triples[i][1]], 
                                                                              tf.entity_to_id[pos_triples[i][2]]]]))
            score_np     = score_triple.detach().numpy()
            score        = (score_np - min_score) / (max_score - min_score)
            score_pos.append(score_np)
            sum_tot      += float(score_np)
            sum_tot_norm += float(score)
        
    if len(diff_rel_triples) > 0:
        for i in range(0, len(diff_rel_triples)):
            score_triple = pipeline_result.model.score_hrt(torch.LongTensor([[tf.entity_to_id[diff_rel_triples[i][0]], 
                                                                              tf.relation_to_id[diff_rel_triples[i][1]], 
                                                                              tf.entity_to_id[diff_rel_triples[i][2]]]]))
            score_np     = score_triple.detach().numpy()
            score        = (score_np - min_score) / (max_score - min_score)
            score_diff_rel.append(score_np)
            sum_tot      += 1 - float(score_np)
            sum_tot_norm += 1 - float(score)

    if len(neg_triples) > 0:
        for i in range(0, len(neg_triples)):
            score_triple = pipeline_result.model.score_hrt(torch.LongTensor([[tf.entity_to_id[neg_triples[i][0]], 
                                                                              tf.relation_to_id[neg_triples[i][1]], 
                                                                              tf.entity_to_id[neg_triples[i][2]]]]))
            score_np     = score_triple.detach().numpy()
            score        = (score_np - min_score) / (max_score - min_score)
            score_neg.append(score_np)
            sum_tot      += 1 - float(score_np)
            sum_tot_norm += 1 - float(score)

    df = pd.DataFrame({'pos': score_pos[:len(score_pos)],
                       'diff_rel': score_diff_rel[:len(score_pos)],
                       'neg': score_neg[:len(score_pos)]},
                      columns=['pos','diff_rel','neg'])

    df.to_csv('scores.csv', mode='a', index=False, header=False)

    sum_tot      = sum_tot / (len(pos_triples) + len(diff_rel_triples) + len(neg_triples))

    sum_tot_norm = sum_tot_norm / (len(pos_triples))

    print("Reliability of the graph -> %.2f" % sum_tot)
    print("Normalized Reliability of the graph -> %.2f" % sum_tot_norm + "\n")
'''

In [ ]:
(20-np.mean([1,2,3,4,5]))/(5*np.std([1,2,3,4,5]))

2.4041630560342613

# LP Classifier

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
 
%matplotlib inline

from sklearn import datasets, metrics, model_selection, svm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.multioutput import ClassifierChain #multiclasse ?
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, plot_confusion_matrix, accuracy_score
from scipy.stats import randint
from sklearn.model_selection import cross_val_score

def accuracy(G, pos_triples, diff_rel_triples, neg_triples):

    ds = [] 
    ds_train = []
    ds_pos = [] 
    ds_neg = []
    ds_all = []

    for t in pos_triples + tf_all[:len(neg_triples)]:
        ds_pos.append([*entity2embedding[t[0]], *relation2embedding[t[1]], *entity2embedding[t[2]], 1])
    for t in diff_rel_triples + neg_triples:
        ds_neg.append([*entity2embedding[t[0]], *relation2embedding[t[1]], *entity2embedding[t[2]], 0])

    ds = ds_pos + ds_neg[:len(ds_pos)]

    dataset = np.array(ds)

    X = dataset[:, :-1]
    y = dataset[:, -1]
    
    # format all fields as string
    #X = X.astype(str)

    # prepare input data
    def prepare_inputs(X_train, X_test):
        ohe = OneHotEncoder(handle_unknown = "ignore")
        ohe.fit(X_train)
        X_train_enc = ohe.transform(X_train)
        X_test_enc = ohe.transform(X_test)
        return X_train_enc, X_test_enc
    
    # prepare target
    def prepare_targets(y_train, y_test):
        le = LabelEncoder()
        le.fit(y_train)
        y_train_enc = le.transform(y_train)
        y_test_enc = le.transform(y_test)
        return y_train_enc, y_test_enc

    # split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
    # prepare input data
    #X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
    # prepare output data
    #y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

    dict_classifiers = {
        "Linear SVM": SVC(),
        "Logistic Regression": LogisticRegression(),
        "Nearest Neighbors": KNeighborsClassifier(),
        "Decision Tree": tree.DecisionTreeClassifier(),
        "Random Forest": RandomForestClassifier(n_estimators=1000,min_samples_leaf=2, min_samples_split=10,
                                                max_features = 'sqrt', max_depth=10,bootstrap=True),
        "Neural Net": MLPClassifier(alpha = 1),
        "Naive Bayes": GaussianNB(),
        "AdaBoost": AdaBoostClassifier(n_estimators=54,learning_rate=0.1),
        "QDA": QuadraticDiscriminantAnalysis(),
        "Gaussian Process": GaussianProcessClassifier(),
        #"Bagging":BaggingClassifier(),
        "BernoulliNB":BernoulliNB,
        "Classifier Chain":CalibratedClassifierCV(),
        #"CategoricalNB":CategoricalNB(),
        #"Classifier chain":ClassifierChain(),
        "ComplementNB": ComplementNB(),
        #"Dummy classifier":DummyClassifier,
        "ExtraTree":tree.ExtraTreeClassifier(),
        #"ExtraTrees":tree.ExtraTreesClassifier(),
        "GaussianNB":GaussianNB()

    }

      
    def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers, verbose = True):
      """
      This method, takes as input the X, Y matrices of the Train and Test set.
      And fits them on all of the Classifiers specified in the dict_classifier.
      The trained models, and accuracies are saved in a dictionary. The reason to use a dictionary
      is because it is very easy to save the whole dictionary with the pickle module.
      
      Usually, the SVM, Random Forest and Gradient Boosting Classifier take quiet some time to train. 
      So it is best to train them on a smaller dataset first and 
      decide whether you want to comment them out or not based on the test accuracy score.
      """
      
      dict_models = {}
      for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        t_start = time.process_time()
        classifier.fit(X_train, Y_train)
        t_end = time.process_time()

        metrics.plot_roc_curve(classifier, X_test, y_test)
        plt.show()
        
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, y_train)
        test_score = classifier.score(X_test, y_test)
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'train_time': t_diff}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))

      return dict_models

    def display_dict_models(dict_models, sort_by='test_score'):
        cls = [key for key in dict_models.keys()]
        test_s = [dict_models[key]['test_score'] for key in cls]
        training_s = [dict_models[key]['train_score'] for key in cls]
        training_t = [dict_models[key]['train_time'] for key in cls]
        
        df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
        for ii in range(0,len(cls)):
            df_.loc[ii, 'classifier'] = cls[ii]
            df_.loc[ii, 'train_score'] = training_s[ii]
            df_.loc[ii, 'test_score'] = test_s[ii]
            df_.loc[ii, 'train_time'] = training_t[ii]
        
        display(df_.sort_values(by=sort_by, ascending=False))

    dict_models = batch_classify(X_train, y_train, X_test, y_test, no_classifiers = 4)
    display_dict_models(dict_models)


# Reliability/Accuracy of subgraphs

In [ ]:
# TRIPLES FOR THE TRAINING SET OF THE LP CLASSIFIER #

# Number of nodes for each subgraph
k = 20

# Number of subgraphs
h = 10

# List of subgraphs
subgraphs = []

## RANDOM WALK ##

while len(subgraphs) < h:
    # List of visited nodes
    visited = []
    # Random node
    node = random.choice(list(G.nodes()))
    # Add the node to the subgraph
    visited.append(node)
    while len(visited) < k:
        # Neighbors of the chosen entity
        neighbors = list(set(G.neighbors(node)) - set(visited))
        if len(neighbors) == 0:
            neighbors = list(set(G.nodes()) - set(visited))
        # Choose the new node randomly
        node = random.choice(neighbors)
        # Add the node to the subgraph
        visited.append(node)

    # Add the generated subgraph to the list (with optional density)
    #if nx.density(nx.subgraph(G, visited)) > 0.6 or nx.density(nx.subgraph(G, visited)) < 0.4:
    #    print(nx.density(nx.subgraph(G, visited)))
    #    print(visited)
        subgraphs.append(visited)

rel_scores      = []
rel_scores_norm = []
rel_scores_sigm = []
acc_scores      = []

perc_entities   = []

# Lists used to evaluate the accuracy on a subgraph
pos_subgraph_triples_sub      = []
diff_rel_subgraph_triples_sub = []
neg_subgraph_triples_sub      = []

# Lists to keep track of all the generated triples
pos_subgraph_triples_tot      = []
diff_rel_subgraph_triples_tot = []
neg_subgraph_triples_tot      = []

c = 0

for s in subgraphs:

    # Lists of the generated triples of the current subgraph
    pos_subgraph_triples      = []
    diff_rel_subgraph_triples = []
    neg_subgraph_triples      = []

    # Compute all the combinations of e-r-e
    for comb in itertools.combinations(s, 2):
        #print(comb)
        n1 = n2 = 0
        # Check if exists at least one relation between the current pair of entities
        if G.get_edge_data(comb[0], comb[1]) != None:
            # Save all the relations between the current pair of entities
            edge_data = [value['label'] for key, value in G[comb[0]][comb[1]].items()]
            # Consider the remaining relations with the set difference
            diff_relations = set(relations) - set(edge_data)
            # Add all the 'positive' triples
            for r in edge_data:
                if [comb[0], r, comb[1]] not in pos_subgraph_triples:
                    pos_subgraph_triples.append([comb[0], r, comb[1]])
            # Add all the 'different relation' triples
            for r in diff_relations:
                diff_rel_subgraph_triples.append([comb[0], r, comb[1]])
        else:
            # If there are no relations between the current pair of entities
            # Add all the 'negative' triples
            for r in relations:
                neg_subgraph_triples.append([comb[0], r, comb[1]])

        #print(n1, n2)

    if pos_subgraph_triples:
        pos_subgraph_triples_tot.extend(pos_subgraph_triples)

    if diff_rel_subgraph_triples:
        diff_rel_subgraph_triples_tot.extend(diff_rel_subgraph_triples)

    if neg_subgraph_triples:
        neg_subgraph_triples_tot.extend(neg_subgraph_triples)

# Compose the train dataset for the LP classifier
ds_pos = []
ds_neg = []
ds_tot = []

for t in pos_subgraph_triples_tot:
    ds_pos.append([*entity2embedding[t[0]], *relation2embedding[t[1]], *entity2embedding[t[2]], 1])
for t in diff_rel_subgraph_triples_tot + neg_subgraph_triples_tot:
    ds_neg.append([*entity2embedding[t[0]], *relation2embedding[t[1]], *entity2embedding[t[2]], 0])

ds_tot = ds_pos + ds_neg[:len(ds_pos)]

#train_dataset = np.array(ds_tot)

## Undirected

In [ ]:
# UNDIRECTED GRAPH #

# To obtain the running time of this task
start_time = time.time()

# Number of nodes for each subgraph
k = 30

# Number of subgraphs
h = 100

# List of subgraphs
subgraphs = []

## RANDOM WALK ##

while len(subgraphs) < h:
    # List of visited nodes
    visited = []
    # Random node
    node = random.choice(list(G.nodes()))
    # Add the node to the subgraph
    visited.append(node)
    while len(visited) < k:
        # Neighbors of the chosen entity
        neighbors = list(set(G.neighbors(node)) - set(visited))
        if len(neighbors) == 0:
            neighbors = list(set(G.nodes()) - set(visited))
        # Choose the new node randomly
        node = random.choice(neighbors)
        # Add the node to the subgraph
        visited.append(node)

    # Add the generated subgraph to the list (with optional density)
    #if nx.density(nx.subgraph(G, visited)) > 0.6 or nx.density(nx.subgraph(G, visited)) < 0.4:
    #    print(nx.density(nx.subgraph(G, visited)))
    #    print(visited)
    subgraphs.append(visited)

rel_scores      = []
rel_scores_norm = []
rel_scores_sigm = []
acc_scores      = []

perc_entities   = []

# Lists used to evaluate the accuracy on a subgraph
pos_subgraph_triples_sub      = []
diff_rel_subgraph_triples_sub = []
neg_subgraph_triples_sub      = []

# Lists to keep track of all the generated triples
pos_subgraph_triples_tot      = []
diff_rel_subgraph_triples_tot = []
neg_subgraph_triples_tot      = []

c = 0

for s in subgraphs:

    c += 1
    
    print("*** Subgraph " + str(c) + " ***\n")

    # Lists of the generated triples of the current subgraph
    pos_subgraph_triples      = []
    diff_rel_subgraph_triples = []
    neg_subgraph_triples      = []

    # Compute all the combinations of e-r-e
    for comb in itertools.combinations(s, 2):
        #print(comb)
        n1 = n2 = 0
        # Check if exists at least one relation between the current pair of entities
        if G.get_edge_data(comb[0], comb[1]) != None:
            # Save all the relations between the current pair of entities
            edge_data = [value['label'] for key, value in G[comb[0]][comb[1]].items()]
            # Consider the remaining relations with the set difference
            diff_relations = set(relations) - set(edge_data)
            # Add all the 'positive' triples
            for r in edge_data:
                if [comb[0], r, comb[1]] not in pos_subgraph_triples:
                    pos_subgraph_triples.append([comb[0], r, comb[1]])
            # Add all the 'different relation' triples
            for r in diff_relations:
                diff_rel_subgraph_triples.append([comb[0], r, comb[1]])
        else:
            # If there are no relations between the current pair of entities
            # Add all the 'negative' triples
            for r in relations:
                neg_subgraph_triples.append([comb[0], r, comb[1]])

        #print(n1, n2)
    
    print(nx.subgraph(G, s))
    
    reliability(s, pos_subgraph_triples, diff_rel_subgraph_triples, neg_subgraph_triples)
    accuracy(s, pos_subgraph_triples, diff_rel_subgraph_triples, neg_subgraph_triples)

    for e in s:
        if e not in perc_entities:
            perc_entities.append(e)
'''
    if pos_subgraph_triples:
        pos_subgraph_triples_sub.append(pos_subgraph_triples)
        pos_subgraph_triples_tot.extend(pos_subgraph_triples)

    if diff_rel_subgraph_triples:
        diff_rel_subgraph_triples_sub.append(diff_rel_subgraph_triples)
        diff_rel_subgraph_triples_tot.extend(diff_rel_subgraph_triples)

    if neg_subgraph_triples:
        neg_subgraph_triples_sub.append(neg_subgraph_triples)
        neg_subgraph_triples_tot.extend(neg_subgraph_triples)

    

# Compose the train dataset for the LP classifier
ds_pos = []
ds_neg = []
ds_tot = []

for t in pos_subgraph_triples_tot:
    ds_pos.append([*entity2embedding[t[0]], *relation2embedding[t[1]], *entity2embedding[t[2]], 1])
for t in diff_rel_subgraph_triples_tot + neg_subgraph_triples_tot:
    ds_neg.append([*entity2embedding[t[0]], *relation2embedding[t[1]], *entity2embedding[t[2]], 0])

ds_tot = ds_pos + ds_neg[:len(ds_pos)]

#train_dataset = np.array(ds_tot)

print(len(subgraphs), len(pos_subgraph_triples_sub), len(diff_rel_subgraph_triples_sub), len(neg_subgraph_triples_sub))

for i in range(0, len(subgraphs)):
    accuracy(ds_tot, pos_subgraph_triples_sub[i], 
             diff_rel_subgraph_triples_sub[i], neg_subgraph_triples_sub[i])

'''


entity_difference = [item for item in entities if item not in perc_entities]

print(1 - len(entity_difference) / len(entities), len(entity_difference), len(entities))
    

df = pd.read_csv('stats.csv', index_col=[0])
df_total = df.assign(**{dataset + "_" + str(k) + "_" + str(h): rel_scores, 
                        dataset + "_" + str(k) + "_" + str(h) + "_NORM": rel_scores_norm, 
                        dataset + "_" + str(k) + "_" + str(h) + "_SIGM": rel_scores_sigm,
                        dataset + "_" + str(k) + "_" + str(h) + "_ACC": acc_scores})

df_total.to_csv('stats.csv')


print(rel_scores)
print(rel_scores_norm)
print(rel_scores_sigm)
print(acc_scores)


print("--- %s seconds TOTAL ---\n" % (time.time() - start_time))

*** Subgraph 1 ***

MultiGraph with 30 nodes and 66 edges
Positive:  66
Negative:  528
Different relation: 7236

Mean and std before: -7.5665728222516915, 1.7236334366612514
-2.2953618 -12.984095
Mean and std after: -7.584829915061952, 1.4931137853218681
-4.120717 -11.0112
Reliability of subgraph              -> -8.19
Normalized Reliability of subgraph   -> 0.59
Sigmoid Reliability of subgraph      -> 0.66

Normalized Reliability of subgraph (without diff_rel)   -> 0.57
Normalized Reliability (end)         -> 0.54
Sigmoid Reliability (end)            -> 0.54

--- 1.4900763034820557 seconds RELIABILITY ---

Accuracy of subgraph ->  90.91 %

trained Linear SVM in 0.00 s


,classifier,train_score,test_score,train_time
0,Linear SVM,0.920455,0.909091,0.00169


*** Subgraph 2 ***

MultiGraph with 30 nodes and 66 edges
Positive:  58
Negative:  536
Different relation: 7236

Mean and std before: -7.567452833813878, 1.7216348836615862
-2.2887826 -12.984095
Mean and std after: -7.587210041173915, 1.4948342099693432
-4.125046 -11.010347
Reliability of subgraph              -> -8.04
Normalized Reliability of subgraph   -> 0.57
Sigmoid Reliability of subgraph      -> 0.62

Normalized Reliability of subgraph (without diff_rel)   -> 0.56
Normalized Reliability (end)         -> 0.55
Sigmoid Reliability (end)            -> 0.55

--- 1.448333978652954 seconds RELIABILITY ---

Accuracy of subgraph ->  89.74 %

trained Linear SVM in 0.00 s


,classifier,train_score,test_score,train_time
0,Linear SVM,0.948052,0.897436,0.004464


*** Subgraph 3 ***

MultiGraph with 30 nodes and 54 edges
Positive:  47
Negative:  439
Different relation: 7344

Mean and std before: -7.572820797520731, 1.7202739351509841
-2.3283696 -12.984095
Mean and std after: -7.595962994970301, 1.4908436292176892
-4.135048 -11.009111
Reliability of subgraph              -> -8.25
Normalized Reliability of subgraph   -> 0.60
Sigmoid Reliability of subgraph      -> 0.67

Normalized Reliability of subgraph (without diff_rel)   -> 0.59
Normalized Reliability (end)         -> 0.55
Sigmoid Reliability (end)            -> 0.56

--- 1.4844584465026855 seconds RELIABILITY ---

Accuracy of subgraph ->  87.50 %

trained Linear SVM in 0.00 s


...

*** Subgraph 49 ***

MultiGraph with 30 nodes and 66 edges
Positive:  58
Negative:  536
Different relation: 7236

Mean and std before: -7.5627427065205, 1.722962014845065
-2.3275867 -12.902369
Mean and std after: -7.584448218778315, 1.4957377715820006
-4.120717 -11.0047035
Reliability of subgraph              -> -7.85
Normalized Reliability of subgraph   -> 0.55
Sigmoid Reliability of subgraph      -> 0.59

Normalized Reliability of subgraph (without diff_rel)   -> 0.53
Normalized Reliability (end)         -> 0.53
Sigmoid Reliability (end)            -> 0.52

--- 1.5341370105743408 seconds RELIABILITY ---

Accuracy of subgraph ->  94.87 %

trained Linear SVM in 0.00 s


,classifier,train_score,test_score,train_time
0,Linear SVM,0.974026,0.948718,0.00136


*** Subgraph 50 ***

MultiGraph with 30 nodes and 80 edges
Positive:  80
Negative:  640
Different relation: 7110

Mean and std before: -7.568826126230174, 1.7204860256619972
-2.3275867 -12.984095
Mean and std after: -7.587929683486418, 1.49226887040162
-4.133936 -11.009111
Reliability of subgraph              -> -7.95
Normalized Reliability of subgraph   -> 0.55
Sigmoid Reliability of subgraph      -> 0.58

Normalized Reliability of subgraph (without diff_rel)   -> 0.53
Normalized Reliability (end)         -> 0.52
Sigmoid Reliability (end)            -> 0.51

--- 1.764153003692627 seconds RELIABILITY ---

Accuracy of subgraph ->  96.23 %

trained Linear SVM in 0.00 s


,classifier,train_score,test_score,train_time
0,Linear SVM,0.925234,0.962264,0.002021


0.034120606697115474 39546 40943
[-8.191, -8.039, -8.249, -7.864, -8.061, -8.068, -8.095, -8.562, -7.971, -8.287, -7.938, -8.327, -8.376, -8.274, -8.067, -8.398, -8.522, -7.624, -8.18, -8.285, -7.882, -8.208, -8.016, -7.929, -8.177, -8.173, -7.717, -8.269, -8.164, -7.986, -7.925, -7.644, -8.308, -7.743, -7.759, -8.236, -8.412, -8.142, -8.335, -7.988, -8.477, -7.923, -8.233, -8.193, -8.395, -8.002, -8.069, -8.185, -7.85, -7.949]
[0.593, 0.573, 0.602, 0.55, 0.579, 0.579, 0.581, 0.645, 0.561, 0.61, 0.559, 0.61, 0.623, 0.609, 0.579, 0.624, 0.638, 0.515, 0.592, 0.609, 0.553, 0.599, 0.57, 0.557, 0.593, 0.593, 0.527, 0.607, 0.593, 0.565, 0.556, 0.517, 0.609, 0.535, 0.534, 0.604, 0.627, 0.588, 0.618, 0.568, 0.635, 0.56, 0.601, 0.59, 0.622, 0.569, 0.57, 0.594, 0.548, 0.555]
[0.655, 0.622, 0.673, 0.595, 0.641, 0.638, 0.633, 0.734, 0.598, 0.691, 0.602, 0.679, 0.698, 0.685, 0.646, 0.692, 0.722, 0.539, 0.664, 0.691, 0.597, 0.662, 0.618, 0.593, 0.651, 0.661, 0.558, 0.686, 0.655, 0.612, 0.593, 0.534,

## Undirected BFS

In [ ]:
# UNDIRECTED GRAPH #

# To obtain the running time of this task
start_time = time.time()

# Number of nodes for each subgraph
k = 30

# Number of subgraphs
h = 100

# List of subgraphs
subgraphs = []

all_visited = []

## RANDOM WALK ##

while len(subgraphs) < h:
    # List of visited nodes
    visited = []
    # Random node
    node = random.choice(list(G.nodes()))
    # Add the node to the subgraph
    visited.append(node)
    all_visited.append(node)
    while len(visited) < k:
        # Neighbors of the chosen entity
        neighbors = list(set(G.neighbors(node)) - set(all_visited))
        if len(neighbors) == 0:
            neighbors = list(set(G.nodes()) - set(visited))
        # Add neighbors to visited nodes
        for n in neighbors:
            if n not in visited and len(visited) < k:
                # Add the node to the subgraph
                visited.append(n)
                all_visited.append(n)

        node = random.choice(visited)

    # Add the generated subgraph to the list (with optional density)
    #if nx.density(nx.subgraph(G, visited)) > 0.6 or nx.density(nx.subgraph(G, visited)) < 0.4:
    #    print(nx.density(nx.subgraph(G, visited)))
    #    print(visited)
    subgraphs.append(visited)

rel_scores      = []
rel_scores_norm = []
rel_scores_sigm = []
acc_scores      = []

perc_entities   = []

# Lists used to evaluate the accuracy on a subgraph
pos_subgraph_triples_sub      = []
diff_rel_subgraph_triples_sub = []
neg_subgraph_triples_sub      = []

# Lists to keep track of all the generated triples
pos_subgraph_triples_tot      = []
diff_rel_subgraph_triples_tot = []
neg_subgraph_triples_tot      = []

c = 0

for s in subgraphs:

    c += 1
    
    print("*** Subgraph " + str(c) + " ***\n")

    # Lists of the generated triples of the current subgraph
    pos_subgraph_triples      = []
    diff_rel_subgraph_triples = []
    neg_subgraph_triples      = []

    # Compute all the combinations of e-r-e
    for comb in itertools.combinations(s, 2):
        #print(comb)
        n1 = n2 = 0
        # Check if exists at least one relation between the current pair of entities
        if G.get_edge_data(comb[0], comb[1]) != None:
            # Save all the relations between the current pair of entities
            edge_data = [value['label'] for key, value in G[comb[0]][comb[1]].items()]
            # Consider the remaining relations with the set difference
            diff_relations = set(relations) - set(edge_data)
            # Add all the 'positive' triples
            for r in edge_data:
                if [comb[0], r, comb[1]] not in pos_subgraph_triples:
                    pos_subgraph_triples.append([comb[0], r, comb[1]])
            # Add all the 'different relation' triples
            for r in diff_relations:
                diff_rel_subgraph_triples.append([comb[0], r, comb[1]])
        else:
            # If there are no relations between the current pair of entities
            # Add all the 'negative' triples
            for r in relations:
                neg_subgraph_triples.append([comb[0], r, comb[1]])
    
    print(nx.subgraph(G, s))
    
    reliability(s, pos_subgraph_triples, diff_rel_subgraph_triples, neg_subgraph_triples)
    accuracy(s, pos_subgraph_triples, diff_rel_subgraph_triples, neg_subgraph_triples)

    for e in s:
        if e not in perc_entities:
            perc_entities.append(e)

entity_difference = [item for item in entities if item not in perc_entities]

print(1 - len(entity_difference) / len(entities), len(entity_difference), len(entities))
    

df = pd.read_csv('stats.csv', index_col=[0])
df_total = df.assign(**{dataset + "_" + str(k) + "_" + str(h): rel_scores, 
                        dataset + "_" + str(k) + "_" + str(h) + "_NORM": rel_scores_norm, 
                        dataset + "_" + str(k) + "_" + str(h) + "_SIGM": rel_scores_sigm,
                        dataset + "_" + str(k) + "_" + str(h) + "_ACC": acc_scores})

df_total.to_csv('stats.csv')


print(rel_scores)
print(rel_scores_norm)
print(rel_scores_sigm)
print(acc_scores)


print("--- %s seconds TOTAL ---\n" % (time.time() - start_time))

## Directed

In [ ]:
# DIRECTED GRAPH #

# To obtain the running time of this task
start_time = time.time()

# Number of nodes for each subgraph
k = 30

# Number of subgraphs
h = 50

# List of subgraphs
subgraphs = []

## RANDOM WALK ##

while len(subgraphs) < h:
    # List of visited nodes
    visited = []
    # Random node
    node = random.choice(list(G.nodes()))
    # Add the node to the subgraph
    visited.append(node)
    while len(visited) < k:
        # Neighbors of the chosen entity
        neighbors = list(set(G.neighbors(node)) - set(visited))
        if len(neighbors) == 0:
            neighbors = list(set(G.nodes()) - set(visited))
        # Choose the new node randomly
        node = random.choice(neighbors)
        # Add the node to the subgraph
        visited.append(node)

    # Add the generated subgraph to the list (with optional density)
    #if nx.density(nx.subgraph(G, visited)) > 0.1 or nx.density(nx.subgraph(G, visited)) < 0.06:
    #    print(nx.density(nx.subgraph(G, visited)))
    #    print(visited)
    subgraphs.append(visited)

rel_scores      = []
rel_scores_norm = []
rel_scores_sigm = []
rel_scores_sigm_end = []
acc_scores      = []

perc_entities   = []

c = 0

for s in subgraphs:

    c += 1
    
    print("*** Subgraph " + str(c) + " ***\n")

    #print(s)
    #print(s[::-1])

    pos_subgraph_triples      = []
    diff_rel_subgraph_triples = []
    neg_subgraph_triples      = []

    edge_data_g   = []
    edge_data_rev = []

    # Compute all the combinations of e-r-e (only in one direction)
    for comb in itertools.combinations(s, 2):
        #print(comb)
        n1 = n2 = 0
        # Check if exists at least one relation between the current pair of entities
        if G.get_edge_data(comb[0], comb[1]) != None:
            # Save all the relations between the current pair of entities
            edge_data = [value['label'] for key, value in G[comb[0]][comb[1]].items()]
            # Merge the sets
            #edge_data = list(set(edge_data_g).union(set(edge_data_rev)))
            # Consider the remaining relations with the set difference
            diff_relations = set(relations) - set(edge_data)
            # Add all the 'positive' triples
            for r in edge_data:
                n1 += 1
                pos_subgraph_triples.append([comb[0], r, comb[1]])
            # Add all the 'different relation' triples
            for r in diff_relations:
                #n2 += 1
                diff_rel_subgraph_triples.append([comb[0], r, comb[1]])
        else:
            # If there are no relations between the current pair of entities
            # Add all the 'negative' triples
            for r in relations:
                neg_subgraph_triples.append([comb[0], r, comb[1]])

    # Compute all the combinations of e-r-e (in the opposite direction)
    for comb in itertools.combinations(s[::-1], 2):
        #print(comb)
        n1 = n2 = 0
        # Check if exists at least one relation between the current pair of entities
        if G.get_edge_data(comb[0], comb[1]) != None:
            # Save all the relations between the current pair of entities
            edge_data = [value['label'] for key, value in G[comb[0]][comb[1]].items()]
            # Merge the sets
            #edge_data = list(set(edge_data_g).union(set(edge_data_rev)))
            # Consider the remaining relations with the set difference
            diff_relations = set(relations) - set(edge_data)
            # Add all the 'positive' triples
            for r in edge_data:
                n1 += 1
                pos_subgraph_triples.append([comb[0], r, comb[1]])
            # Add all the 'different relation' triples
            for r in diff_relations:
                #n2 += 1
                diff_rel_subgraph_triples.append([comb[0], r, comb[1]])
        else:
            # If there are no relations between the current pair of entities
            # Add all the 'negative' triples
            for r in relations:
                neg_subgraph_triples.append([comb[0], r, comb[1]])
    
    print(nx.subgraph(G, s))
    
    reliability(s, pos_subgraph_triples, diff_rel_subgraph_triples, neg_subgraph_triples)
    accuracy(s, pos_subgraph_triples, diff_rel_subgraph_triples, neg_subgraph_triples)

    for e in s:
        if e not in perc_entities:
            perc_entities.append(e)

entity_difference = [item for item in entities if item not in perc_entities]

print(1 - len(entity_difference) / len(entities), len(entity_difference), len(entities))
    

df = pd.read_csv('stats.csv', index_col=[0])
df_total = df.assign(**{dataset + "_" + str(k) + "_" + str(h): rel_scores, 
                        dataset + "_" + str(k) + "_" + str(h) + "_NORM": rel_scores_norm, 
                        dataset + "_" + str(k) + "_" + str(h) + "_SIGM": rel_scores_sigm,
                        dataset + "_" + str(k) + "_" + str(h) + "_SIGMEND": rel_scores_sigm_end,
                        dataset + "_" + str(k) + "_" + str(h) + "_ACC": acc_scores})

df_total.to_csv('stats.csv')

print("--- %s seconds TOTAL ---\n" % (time.time() - start_time))

# Reliability/Accuracy of the graph

In [ ]:
df = pd.read_csv("stats.csv")
df[dataset + "_total"] = ""
df.to_csv("stats.csv", index=False)

#total_reliability(entities, pos_triples, diff_rel_triples, neg_triples)
accuracy(entities, pos_triples, diff_rel_triples, neg_triples)

# Compute scores

In [ ]:
df = pd.DataFrame(list())
df.to_csv("scores.csv")

if len(pos_triples) > 0:
    for i in range(0, len(pos_triples)):
        score_triple = model.score_hrt(torch.LongTensor([[tf.entity_to_id[pos_triples[i][0]], 
                                                                            tf.relation_to_id[pos_triples[i][1]], 
                                                                            tf.entity_to_id[pos_triples[i][2]]]]))
        score_np = score_triple.detach().numpy()
        score_pos.append(score_np)
        
if len(diff_rel_triples) > 0:
    for i in range(0, len(diff_rel_triples)):
        score_triple = model.score_hrt(torch.LongTensor([[tf.entity_to_id[diff_rel_triples[i][0]], 
                                                                            tf.relation_to_id[diff_rel_triples[i][1]], 
                                                                            tf.entity_to_id[diff_rel_triples[i][2]]]]))
        score_np = score_triple.detach().numpy()
        score_diff_rel.append(score_np)

if len(neg_triples) > 0:
    for i in range(0, len(neg_triples)):
        score_triple = model.score_hrt(torch.LongTensor([[tf.entity_to_id[neg_triples[i][0]], 
                                                                            tf.relation_to_id[neg_triples[i][1]], 
                                                                            tf.entity_to_id[neg_triples[i][2]]]]))
        score_np = score_triple.detach().numpy()
        score_neg.append(score_np)
else:
    score_neg = score_diff_rel

min_len = min(len(score_pos), len(score_diff_rel), len(score_neg))

df = pd.DataFrame({'pos': score_pos[:min_len],
                    'diff_rel': score_diff_rel[:min_len],
                    'neg': score_neg[:min_len]},
                    columns=['pos','diff_rel','neg'])

df.to_csv('scores.csv', mode='a', index=False, header=False)

In [ ]:
df = pd.DataFrame(list())
df.to_csv("scores.csv")

new_score_pos = []
new_score_diff_rel = []
new_score_neg = []

for e in score_pos:
    new_score_pos.append(e.tolist()[0][0])

for e in score_diff_rel:
    new_score_diff_rel.append(e.tolist()[0][0])

for e in score_neg:
    new_score_neg.append(e.tolist()[0][0])

min_len = min(len(new_score_pos), len(new_score_diff_rel), len(new_score_neg))

df = pd.DataFrame({'pos': new_score_pos[:min_len],
                    'diff_rel': new_score_diff_rel[:min_len],
                    'neg': new_score_neg[:min_len]},
                    columns=['pos','diff_rel','neg'])

df.to_csv('scores.csv', mode='a', index=False, header=False)

# Plot scores

In [ ]:
a = np.loadtxt(r'scores.csv',dtype=str,delimiter=',',skiprows=1,usecols=(0,1,2))

pos = a[:,0]

pos_elem = []
for i in pos:
    pos_elem.append(eval(i)[0][0])

diff_rel = a[:,1]
diff_rel_elem = []
for i in diff_rel:
    diff_rel_elem.append(eval(i)[0][0])

neg = a[:,2]
neg_elem = []
for i in neg:
    neg_elem.append(eval(i)[0][0])

sns.distplot(pos_elem, hist=False, rug=True)
sns.distplot(diff_rel_elem, hist=False, rug=True)
sns.distplot(neg_elem, hist=False, rug=True)
plt.legend(labels=["Positive","Different relation","Negative"], fontsize='small')
plt.xlabel("Score")
plt.show()

print("** Mean **")
print("Positive: " + str(statistics.mean(pos_elem)))
print("Different relation: " + str(statistics.mean(diff_rel_elem)))
print("Negative: " + str(statistics.mean(neg_elem)) + "\n")

print("** Standard deviation **")
print("Positive: " + str(statistics.stdev(pos_elem)))
print("Different relation: " + str(statistics.stdev(diff_rel_elem)))
print("Negative: " + str(statistics.stdev(neg_elem)))


# Evaluate triples

In [ ]:
df = get_relation_prediction_df(pipeline_result.model, entities[0], entities[1], triples_factory=pipeline_result.training)

score_trip = pipeline_result.model.score_hrt(torch.LongTensor([[tf.entity_to_id[entities[0]], tf.relation_to_id['affects'], tf.entity_to_id[entities[1]]]]))
result_np = score_trip.detach().numpy()
#df = get_relation_prediction_df(pipeline_result.model, 'a-51', 'a-52', triples_factory=pipeline_result.training)
print(df)

print(tf.relation_to_id['affects'])
#print(score_trip, result_np)

# Evaluation metrics

In [ ]:
# Precision@k (fixed k)
def pk(r_list, k):
    
    #print(r_list[:k])

    num_true = r_list[:k].count(True)
    
    p = 0
    
    if num_true == 0:
        return 1

    for e in r_list[:k]:
        if e == True:
            p += 1
        elif p >= num_true:
            p += 1

    precision = p / len(r_list[:k])
    
    return float("{:.2f}".format(precision))

# Precision@k (variable k)
def variable_pk(r_list):

    k = r_list.count(True)
    
    p = 0
    
    if k == 0:
        return 1

    for e in r_list[:k]:
        if e == True:
            p += 1
        elif p >= k:
            p += 1

    precision = p / len(r_list[:k])
    
    return float("{:.2f}".format(precision))


# Mean Reciprocal Rank
def mrr(r_list):

    k = 0

    if r_list.count(True) > 0:
        k = len(r_list) - r_list[::-1].index(True)
    else:
        return 1

    for i in range(0, len(r_list[:k])):
        if r_list[i] == True:
            mrr = 1 / (i + 1)
            return float("{:.2f}".format(mrr))

# Evaluation of subgraph A

In [ ]:
positives = [0, 0]
all_scores = [0, 0]
to_append_a = []
to_append_b = []

for i in range(1,100):
    for j in range(i,100):
        labels = []
        
        df = get_relation_prediction_df(pipeline_result.model, 'a-' + str(i), 'a-' + str(j), triples_factory=pipeline_result.training)
        labels = df.relation_label.tolist()
        scores = df.score.tolist()
        if(labels[0] == 'a'):
            positives[0] += 1
            all_scores[0] += scores[0]
            all_scores[1] += scores[1]
            to_append_a.append(scores[0])
            to_append_b.append(scores[1])
        else:
            positives[1] += 1
            all_scores[0] += scores[1]
            all_scores[1] += scores[0]
            to_append_a.append(scores[1])
            to_append_b.append(scores[0])

In [ ]:
print("True positives/False positives: " + str(positives))
print("Precision: " + str(positives[0]/(positives[0] + positives[1])) + "\n")
print("Average score of all 'a' edges: " + str(all_scores[0]/5000))
print("Average score of all 'b' edges: " + str(all_scores[1]/5000) + "\n")

plt.hist(to_append_a, bins=82, label="A")
plt.hist(to_append_b, bins=82, label="B")
plt.legend(loc="upper left")
plt.ylabel("Count")
plt.xlabel("Scores")
plt.title("Distribution of scores in the subgraph A")

# Evaluation of the toy graph

In [ ]:
positives = [0, 0]
all_scores = [0, 0]
to_append_a = []
to_append_b = []

for i in range(1,100):
    for j in range(i,100):
        labels = []
        
        df = get_relation_prediction_df(pipeline_result.model, 'a-' + str(i), 'a-' + str(j), triples_factory=pipeline_result.training)
        labels = df.relation_label.tolist()
        scores = df.score.tolist()
        if(labels[0] == 'a'):
            positives[0] += 1
            all_scores[0] += scores[0]
            all_scores[1] += scores[1]
            to_append_a.append(scores[0])
            to_append_b.append(scores[1])
        else:
            positives[1] += 1
            all_scores[0] += scores[1]
            all_scores[1] += scores[0]
            to_append_a.append(scores[1])
            to_append_b.append(scores[0])
        
        df = get_relation_prediction_df(pipeline_result.model, 'b-' + str(i), 'b-' + str(j), triples_factory=pipeline_result.training)
        labels = df.relation_label.tolist()
        scores = df.score.tolist()
        if(labels[0] == 'b'):
            positives[0] += 1
            all_scores[0] += scores[0]
            all_scores[1] += scores[1]
            to_append_a.append(scores[0])
            to_append_b.append(scores[1])
        else:
            positives[1] += 1
            all_scores[0] += scores[1]
            all_scores[1] += scores[0]
            to_append_a.append(scores[1])
            to_append_b.append(scores[0])

In [ ]:
print("True positives/False positives: " + str(positives))
print("Precision: " + str(positives[0]/(positives[0] + positives[1])) + "\n")
print("Average score of all positive edges: " + str(all_scores[0]/10000))
print("Average score of all negative edges: " + str(all_scores[1]/10000) + "\n")

plt.hist(to_append_a, bins=82, label="A")
plt.hist(to_append_b, bins=82, label="B")
plt.legend(loc="upper left")
plt.ylabel("Count")
plt.xlabel("Scores")
plt.title("Distribution of scores in the toy graph")

# Evaluation of the UMLS graph

In [ ]:
# VERSION 1 #

positives       = [0, 0] # True positives/False positives
pos_scores      = []
negatives       = [0, 0] # True negatives/False negatives
neg_scores      = []
scores_count    = 0

for e1 in range(0, len(entities)):
    for e2 in range(e1 + 1, len(entities)):
        df = get_relation_prediction_df(pipeline_result.model, str(entities[e1]), str(entities[e2]), triples_factory=pipeline_result.training)
        labels = df.relation_label.tolist()
        scores = df.score.tolist()
        in_training = df.in_training.tolist()

        for i in range(0, len(labels)):
            if [entities[e1], labels[i], entities[e2]] in tf_all and in_training[i] == True:
                positives[0] += 1
                pos_scores.append(scores[i]) 
            elif [entities[e1], labels[i], entities[e2]] not in tf_all and in_training[i] == False:
                negatives[0] += 1
                neg_scores.append(scores[i]) 
            elif [entities[e1], labels[i], entities[e2]] in tf_all and in_training[i] == False:
                negatives[1] += 1
            scores_count += 1


In [ ]:
# VERSION 2 # (with P@k, MRR)

positives       = 0     # Number of positive triples 
pos_scores      = []    # List of positives' scores
negatives       = 0     # Number of negative triples
neg_scores      = []    # List of negatives' scores
pairs_count     = 0     # Total pairs of entities
scores_count    = 0     # Total triples (positives/negatives)
prec_at_3       = 0     # Sum of all P@3
prec_at_5       = 0     # Sum of all P@5
prec_at_10      = 0     # Sum of all P@10
var_prec_at_k   = 0     # Sum of all P@k (with variable k)
mrr_value       = 0     # Sum of MRR

for e1 in range(0, len(entities)):
    for e2 in range(e1 + 1, len(entities)):
        df = get_relation_prediction_df(pipeline_result.model, str(entities[e1]), str(entities[e2]), triples_factory=pipeline_result.training)
        labels = df.relation_label.tolist()
        scores = df.score.tolist()
        in_training = df.in_training.tolist()

        ranking_list = []

        for i in range(0, len(labels)):
            if [entities[e1], labels[i], entities[e2]] in tf_all:
                positives += 1
                pos_scores.append(scores[i]) 
                ranking_list.append(True)
            else:
                negatives += 1
                neg_scores.append(scores[i])
                ranking_list.append(False)
            scores_count += 1

        pairs_count += 1

        prec_at_3       += pk(ranking_list, 3)
        prec_at_5       += pk(ranking_list, 5)
        prec_at_10      += pk(ranking_list, 10)
        var_prec_at_k   += variable_pk(ranking_list)
        mrr_value       += mrr(ranking_list)

        


In [ ]:
#print("Precision: " + str(positives[0]/(positives[0] + positives[1])))

#print("Recall: " + str(positives[0]/(positives[0] + negatives[1])) + "\n")

print("Precision@3: " + str(float("{:.5f}".format(prec_at_3 / pairs_count))) + "\n")

print("Precision@5: " + str(float("{:.5f}".format(prec_at_5 / pairs_count))) + "\n")

print("Precision@10: " + str(float("{:.5f}".format(prec_at_10 / pairs_count))) + "\n")

print("Variable Precision@k: " + str(float("{:.5f}".format(var_prec_at_k / pairs_count))) + "\n")

print("MRR: " + str(float("{:.5f}".format(mrr_value / pairs_count))))

#print("True Positives/False Positives: " + str(positives))
#print("True Negatives/False Negatives: " + str(negatives))

In [ ]:
plt.hist(pos_scores, bins=50, label="Pos")
plt.legend(loc="upper left")
plt.ylabel("Count")
plt.xlabel("Scores")
plt.title("Positive triples")

In [ ]:
plt.hist(neg_scores, bins=50, label="Neg")
plt.legend(loc="upper left")
plt.ylabel("Count")
plt.xlabel("Scores")
plt.title("Negative triples")

# Empty the stats document

In [ ]:
df = pd.DataFrame(list())
df.to_csv("stats.csv")